## NetCDF basics in Python

1. Read data from a NetCDF file
2. Create a simple contour plot

In [1]:
import os, sys
import numpy as np
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


### Reading data from a NetCDF file

For this example, we will read monthly 2-meter temperature data from a netCDF file of ERA-Interim reanalysis (0.75 deg, 2000-2005).

In [2]:
# Path to data file
datdir = '../sample-data/'
filename = 'erai.mon.t2m.2000-2005.nc'
filepath = datdir + filename
print(filepath)

# Open netCDF file object
f = nc.Dataset(filepath,'r')

# List file contents (variables, dimensions, global attributes)
print(f)

../sample-data/erai.mon.t2m.2000-2005.nc
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: Thu May  2 00:07:54 2019: ncpdq -U era.t2m.mon.2000.nc out2000.nc
2019-05-02 05:57:01 GMT by grib_to_netcdf-2.12.0: grib_to_netcdf /data/data02/scratch/d8/a7/_mars-atls18-70e05f9f8ba4e9d19932f1c45a7be8d8-4AHW9i.grib -o /data/data02/scratch/f6/3a/_grib2netcdf-atls02-a82bacafb5c306db76464bc7e824bb75-ZBvnn_.nc -utime
    NCO: 4.6.7
    nco_openmp_thread_number: 1
    dimensions(sizes): longitude(480), latitude(241), time(72)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 time(time), float64 t2m(time,latitude,longitude)
    groups: 


In [3]:
# Read data into numpy arrays
lons = f.variables['longitude'][:] # [:] syntax stores data in numpy arrays
lats = f.variables['latitude'][:]
t2m  = f.variables['t2m'][:]


In [31]:
# Retrieve attributes
t2m_units    = f.variables['t2m'].units
t2m_longname = f.variables['t2m'].long_name
t2m_missing  = f.variables['t2m'].missing_value
print('units:', t2m_units)
print(t2m_missing)


idx = (t2m == t2m_missing)
tmp = t2m[idx]
print(tmp)

units: K
-32767
[]


In [6]:
# Print array size/shape
print(t2m.shape)

# Print size of lat/lon arrays...
print(lats.shape)


(72, 241, 480)
(241,)


In [8]:
# Print data values
print(t2m[0,:,:].shape)


(241, 480)


In [11]:
#t2m[:,10,20].shape
lons[20]

15.0

### Time coordinate handling

Need to convert times from netcdf to a datetime array. 


In [ ]:
# Read time from netcdf file
time = f.variables['time'][:]
print(time)


In [ ]:
time_units = f.variables['time'].units
print(time_units)


**Option 1:** Read in times from netcdf file and convert it to a datetime object using the `num2date()` function. You will also need to retrieve in time.units attribute.

In [24]:
# convert times to a datetime object
dates = nc.num2date(time, time_units)
print(dates)

NameError: name 'time' is not defined

**Option 2**: Create a pandas datetime object from scratch using `pd.date_range`

In [25]:
dates_pd = pd.date_range(start='2000-01', end='2006-01', freq='M')
print(dates_pd)

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-30', '2000-10-31', '2000-11-30', '2000-12-31',
               '2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30',
               '2001-05-31', '2001-06-30', '2001-07-31', '2001-08-31',
               '2001-09-30', '2001-10-31', '2001-11-30', '2001-12-31',
               '2002-01-31', '2002-02-28', '2002-03-31', '2002-04-30',
               '2002-05-31', '2002-06-30', '2002-07-31', '2002-08-31',
               '2002-09-30', '2002-10-31', '2002-11-30', '2002-12-31',
               '2003-01-31', '2003-02-28', '2003-03-31', '2003-04-30',
               '2003-05-31', '2003-06-30', '2003-07-31', '2003-08-31',
               '2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31',
               '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30',
               '2004-05-31', '2004-06-30', '2004-07-31', '2004-08-31',
      

### Simple arithmetic operations

In [13]:
# Convert temperature to C
t2m_degC = t2m - 273.15
print(t2m_degC)


(72, 241, 480)
[[[-27.54304121 -27.54304121 -27.54304121 ... -27.54304121 -27.54304121
   -27.54304121]
  [-27.3116225  -27.30982855 -27.30624067 ... -27.31521038 -27.31341644
   -27.3116225 ]
  [-26.977949   -26.97077323 -26.9653914  ... -27.01023999 -26.99768238
   -26.98871266]
  ...
  [-25.9141405  -25.92490416 -25.93387388 ... -25.87467374 -25.88723135
   -25.9015829 ]
  [-26.62992396 -26.63709974 -26.64248157 ... -26.61557241 -26.62095424
   -26.62633608]
  [-27.86953893 -27.86953893 -27.86953893 ... -27.86953893 -27.86953893
   -27.86953893]]

 [[-26.39671131 -26.39671131 -26.39671131 ... -26.39671131 -26.39671131
   -26.39671131]
  [-27.27753757 -27.27036179 -27.26139208 ... -27.30085883 -27.29368306
   -27.28471334]
  [-27.56815642 -27.55201093 -27.53945332 ... -27.62018078 -27.60224134
   -27.58609585]
  ...
  [-39.14267994 -39.15164966 -39.16061938 ... -39.09783135 -39.11397684
   -39.12653445]
  [-39.66651144 -39.66830539 -39.67009933 ... -39.65754173 -39.66112961
   -39.66

In [20]:
t2m_1d = t2m_degC[:,50,60]
print(t2m_1d)

idx = (t2m_1d > 20.0)
t2m_20 = t2m_1d[idx]

print(t2m_20.shape)

[ -6.77814509  -3.72126535   0.0944525   13.82709006  13.28890701
  20.34448683  25.15404938  23.61843374  15.01826855   8.55469009
  -0.85813151  -2.96601513  -3.48930135  -5.94030188  -0.02938784
  15.20666022  17.51326922  20.34992308  28.97137686  22.39699288
  16.94849729   6.07892221   0.55000602 -10.33784646  -6.26580737
  -0.88767546   4.78994316  10.99978215  16.26064516  21.2093925
  28.71821001  21.75063355  18.20730881   6.03299344   0.20382732
 -14.35375281  -7.49663695  -9.01149119  -3.09299089   8.04471071
  20.16002174  16.93479137  23.68470004  22.92903438  16.73926948
   8.99765972   0.45106345  -3.20748569  -6.27648002  -6.44470169
   2.52831346   9.19275471  17.4678289   21.08280509  23.47906901
  25.15233769  18.98361347   8.03846776   1.04474155  -4.69269095
  -5.07869632  -6.66023051  -4.58225061  10.33771597  21.62121773
  21.81270393  24.05025566  24.18145917  19.60883959  11.49017922
   2.52401511  -4.00956503]
(19,)


In [27]:
dates_20 = dates_pd[idx]
dates_20

DatetimeIndex(['2000-06-30', '2000-07-31', '2000-08-31', '2001-06-30',
               '2001-07-31', '2001-08-31', '2002-06-30', '2002-07-31',
               '2002-08-31', '2003-05-31', '2003-07-31', '2003-08-31',
               '2004-06-30', '2004-07-31', '2004-08-31', '2005-05-31',
               '2005-06-30', '2005-07-31', '2005-08-31'],
              dtype='datetime64[ns]', freq=None)

In [29]:
# Average over the time dimension
t2m_mean = np.mean(t2m_degC, axis=0)
print(t2m_mean.shape)


(241, 480)


In [ ]:
# Find min and max values of t2m_mean
tmax = t2m_mean.max()
tmin = t2m_mean.min()
print(tmin, tmax)


## Plotting

Create a filled contour plot of 2-meter temperature. We will use the cartopy and matplotlib packages. 


#### Map projection

When plotting with cartopy, you must specify the source coordinates of your data (**data coordinates**, `datacrs`) and the projection or coordinate system that you want to plot your data in (**map coordinates**, `mapcrs`). 

Always use "PlateCarree" if your data are in lat/lon coordinates. See link below for all available projections.

[Cartopy projection list](https://scitools.org.uk/cartopy/docs/latest/crs/projections.html#cartopy-projections)


In [ ]:
# Set your data coordinates
datacrs = ccrs.PlateCarree()
# Set the map projection
mapcrs = ccrs.PlateCarree()


In [ ]:
# Set up contour levels 
clevs = np.arange(-55, 41, 5)  # (start,stop,step)
print(clevs)


In [ ]:
# Create figure
fig = plt.figure(figsize=(7, 5))

# Add plot axes (an individual plot)
ax = fig.add_subplot(1, 1, 1, projection=mapcrs)

# Set up extent of the map [x0, x1, y0, y1]
ax.set_extent([lons.min(), lons.max(), lats.min(), lats.max()], crs=mapcrs)

# Add map features
ax.add_feature(cfeature.COASTLINE, edgecolor='0.9') #Grayscale colors can be set using 0 (black) to 1 (white)
ax.add_feature(cfeature.BORDERS, edgecolor='0.9')  

# Draw contour plot on the plot axes (ax)
p = ax.contourf(lons, lats, t2m_mean, transform=datacrs,
                levels=clevs,  # contour levels
                cmap='YlOrRd') # colormap

# Add colorbar
cbar = plt.colorbar(p, orientation='horizontal', label='deg C')

# Add plot title
plt.title('Average 2m Temperature (2000-2005)')

# Save to fig as png file
filename = 't2m-map.png'
plt.savefig(filename)

# Show figure
plt.show()

## Challenge Time!

1. Change the colormap of the plot. 

[Hint](https://matplotlib.org/tutorials/colors/colormaps.html)

2. Change the extent of the plot. 

[Hint](https://scitools.org.uk/cartopy/docs/v0.15/matplotlib/geoaxes.html#cartopy.mpl.geoaxes.GeoAxes.set_extent)